In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

f2 = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
# X_sequence_val, X_feature_val, Y_val = pickle.load(f2)
f2.close()

from keras_preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
# total_X_image_train = []
total_Y_train = []
for i in range(0,10):
    f = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    # X_sequence_train, X_feature_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    Y_train = np.array(Y_train)-1
    Y_train = keras.utils.to_categorical(Y_train, num_classes=11)

    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
Y_val = np.array(Y_val)-1


Y_val = keras.utils.to_categorical(Y_val, num_classes=11)

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/keras/preprocessing/sequence.py)

In [3]:
from mix_models import rnn_with_feature

X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=860,
    decay_rate=0.85)

for i in range(1,2):
    model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=10, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


2022-10-27 20:00:26.009168: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-27 20:00:26.009328: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-10-27 20:00:26.621151: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-27 20:00:26.621325: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/10
  1/645 [..............................] - ETA: 2:36:56 - loss: 14.1130 - acc: 0.0625

2022-10-27 20:00:41.766429: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-27 20:00:41.766842: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/645 [..............................] - ETA: 8:10 - loss: 12.3595 - acc: 0.0312   

2022-10-27 20:00:42.501389: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-10-27 20:00:42.503874: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-10-27 20:00:42.593923: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 6079 callback api events and 6076 activity events. 
2022-10-27 20:00:42.782314: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-27 20:00:43.025494: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature1/logs/train/plugins/profile/2022_10_27_20_00_42

2022-10-27 20:00:43.172950: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature1/logs/train/plugins/profile/2022_10_27_20_00_42/dell-Precision-7920-T

...

INFO:tensorflow:Assets written to: models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature1/model_9/assets


INFO:tensorflow:Assets written to: models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature1/model_9/assets


Epoch 10/10
645/645 [==============================] - 40s 62ms/step - loss: 0.7401 - acc: 0.9247 - val_loss: 1.2457 - val_acc: 0.7188


INFO:tensorflow:Assets written to: models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature1/model_10/assets


INFO:tensorflow:Assets written to: models/bagging_random_cadence_0.60_0.30_0.10/rnn_with_feature1/model_10/assets
